In [1]:
using Pkg 

In [17]:
Pkg.add(["StableRNGs", "Plots", "StatsPlots", "YAML", "CondaPkg", "PythonCall"])

   Resolving package versions...
   Installed PythonCall ─ v0.9.13
    Updating `~/gitrepos/utd/hamiltonian-nn-for-aq/Project.toml`
  [992eb4ea] + CondaPkg v0.2.18
  [6099a3de] + PythonCall v0.9.13
    Updating `~/gitrepos/utd/hamiltonian-nn-for-aq/Manifest.toml`
  [992eb4ea] + CondaPkg v0.2.18
  [0f8b85d8] + JSON3 v1.12.0
  [0b3b1443] + MicroMamba v0.1.12
  [fa939f87] + Pidfile v1.3.0
  [6099a3de] + PythonCall v0.9.13
  [66db9d55] + SnoopPrecompile v1.0.3
  [856f2bd8] + StructTypes v1.10.0
  [e17b2a0c] + UnsafePointers v1.0.0
  [f8abcde7] + micromamba_jll v1.4.1+0
Precompiling project...
  ✓ UnsafePointers
  ✓ Pidfile
  ✓ micromamba_jll
  ✓ StructTypes
  ✓ MicroMamba
  ✓ JSON3
  ✓ CondaPkg
  ✓ PythonCall
  8 dependencies successfully precompiled in 14 seconds. 192 already precompiled.


# Package Imports

In [18]:
using StableRNGs
using Plots, StatsPlots
using YAML

for reproducibility, use the `StableRNGs` package. We seed with $42$ because it's the best number... *obviously*.

In [8]:
rng = StableRNG(42)
# A = randn(rng, 10, 10) # instead of randn(10, 10)

StableRNGs.LehmerRNG(state=0x00000000000000000000000000000055)

load in the `.credentials.yml` file

In [27]:
credentials = YAML.load_file(".credentials.yml")

Dict{Any, Any} with 3 entries:
  "influx_token"  => "teamlarytoken"
  "influx_bucket" => "SharedAirDFW"
  "influx_org"    => "MINTS"

Now let's set up our python environment in order to interface with the InfluxDB client as it's more stable and mature. We will do so in a reproducible way using `CondaPkg.jl` and `PythonCall.jl` to make easiily managed python environments via toml files (like our `Project.toml` and `Manifest.toml`) 

In [19]:
using CondaPkg

In [23]:
CondaPkg.add("influxdb-client")

    CondaPkg Found dependencies: /Users/jwaczak/gitrepos/utd/hamiltonian-nn-for-aq/CondaPkg.toml
    CondaPkg Found dependencies: /Users/jwaczak/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


In [24]:
CondaPkg.status()

CondaPkg Status /Users/jwaczak/gitrepos/utd/hamiltonian-nn-for-aq/CondaPkg.toml
Environment
  /Users/jwaczak/gitrepos/utd/hamiltonian-nn-for-aq/.CondaPkg/env
Packages
  influxdb-client v1.36.1


In [22]:
using PythonCall 

In [25]:
influxdb_client = pyimport("influxdb_client")

Python: <module 'influxdb_client' from '/Users/jwaczak/gitrepos/utd/hamiltonian-nn-for-aq/.CondaPkg/env/lib/python3.11/site-packages/influxdb_client/__init__.py'>

In [44]:
bucket = credentials["influx_bucket"]
org = credentials["influx_org"]
token = credentials["influx_token"]
url="mdash.circ.utdallas.edu:8086"


client = influxdb_client.InfluxDBClient(
    url=url,
    org=org,
    token=token,
)

Python: <influxdb_client.client.influxdb_client.InfluxDBClient object at 0x29356f290>

perform a sample query to make sure this is working

In [45]:
query_api = client.query_api()

Python: <influxdb_client.client.query_api.QueryApi object at 0x293574650>

In [46]:
query = """
import \"influxdata/influxdb/schema\"

schema.tagValues(bucket: \"$bucket\", tag: \"device_name\")
"""

println(query)

import "influxdata/influxdb/schema"

schema.tagValues(bucket: "SharedAirDFW", tag: "device_name")



In [62]:
result = query_api.query(org=org, query=query)
table = result[0]
for record ∈ table
    println(record.get_value())
end


# see this example: https://docs.influxdata.com/influxdb/cloud/api-guide/client-libraries/python/

Active Pure Node
Central Hub 10
Central Hub 4
Central Hub 5
Central Hub 6
Central Hub 7
Central Hub 8
Central Hub 9
LoRa Node 006
LoRa Node 008
LoRa Node 014
LoRa Node 022
LoRa Node 024
LoRa Node 028
LoRa Node 037
LoRa Node 039
LoRa Node 040
LoRa Node 041
LoRa Node 052
LoRa Node 054
LoRa Node 058
LoRa Node 063
LoRa Node 068
LoRa Node 069
LoRa Node 070
LoRa Node 071
LoRa Node 074
LoRa Node 076
LoRa Node 081
LoRa Node 085
LoRa Node 086
LoRa Node 088
LoRa Node 089
LoRa Node 090
LoRa Node 093
LoRa Node 094
LoRa Node 097
LoRa Node 099
LoRa Node 101
LoRa Node 102
LoRa Node 105
LoRa Node 107
LoRa Node 109
LoRa Node 111
Mints Wearable Node 02
Mints Wearable Node V5 06
PoLo Node 01
PoLo Node 03
PoLo Node 13
PoLo Node 16
PoLo Node 17
PoLo Node 18
PoLo Node 19
PoLo Node 20
PoLo Node 21
UTD Node 1
UTD Node 13
UTD Node 14
UTD Node 15
UTD Node 19
UTD Node 20
UTD Node 4
UTD Node 5
UTD Node 7
WSTC Test Node 01
